Search using Top Charts and HipHopCharts, created connected graph of artists

artist can collaborate with other artists, so we can consider Ingoing and Outgoing links for page rank

Create visual circles describing 

Graph of words for different genres: https://towardsdatascience.com/getting-started-with-graph-analysis-in-python-with-pandas-and-networkx-5e2d2f82f18e

https://networkx.github.io/
https://plot.ly/python/3d-network-graph/

Max distance from one node to another
Avg distance from one node to another
Pagerank


artist = domain
song = document
collaborators = outgoing links / incoming links
lyrics = docuemnt text

In [1]:
from pymongo import MongoClient

# DB CONFIG
client = MongoClient('mongo', 27017)
db = client.music_db

In [10]:
db.collection_names()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['pop-songs',
 'latin-songs',
 'top_songs',
 'country-songs',
 'rock-songs',
 'songs',
 'jazz-songs',
 'all-songs',
 'christian-songs',
 'rap-song']

In [16]:
songs = list(db['songs'].find())
songs[1]

{'_id': ObjectId('5b927851ea311f002198aadb'),
 'title': 'I Like It',
 'artist': 'Cardi B, Bad Bunny & J Balvin',
 'peakPos': 5,
 'lastPos': 0,
 'weeks': 0,
 'rank': 5,
 'isNew': True,
 'identifier': "'I Like It' by Cardi B, Bad Bunny & J Balvin",
 'lyrics': {'error': 'Lyric no found, try again later.'},
 'spotify': {'id': '1m9GAhsNfiLFRzW9pXeoG9',
  'audio_features': {'danceability': 0.778,
   'energy': 0.593,
   'key': 5,
   'loudness': -11.07,
   'mode': 0,
   'speechiness': 0.0956,
   'acousticness': 0.539,
   'instrumentalness': 0.796,
   'liveness': 0.118,
   'valence': 0.559,
   'tempo': 137.013,
   'type': 'audio_features',
   'id': '1m9GAhsNfiLFRzW9pXeoG9',
   'uri': 'spotify:track:1m9GAhsNfiLFRzW9pXeoG9',
   'track_href': 'https://api.spotify.com/v1/tracks/1m9GAhsNfiLFRzW9pXeoG9',
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1m9GAhsNfiLFRzW9pXeoG9',
   'duration_ms': 254928,
   'time_signature': 4}}}

In [48]:
# Filter Collaboration Songs 
SEPARATORS = ['&', 'Featuring', 'With', ',', 'Feat.']

def is_collaboration(artist):
    return any([word in artist for word in SEPARATORS])

collaborations = songs #[song for song in songs if is_collaboration(song['artist'])]

In [86]:

import re
from typing import List

def clean(collaborators: List) -> List:
    return [artist.strip() for artist in collaborators]

def separate(artist: str) -> List:
    return re.split('|'.join(SEPARATORS), artist)

def add_collaborators_to_songs(collaborators, songs):
    for collabs, song in zip(collaborators, songs):
        song['collaborators'] = collabs
    return songs

collaborators = [song['artist'] for song in collaborations]
collaborators = map(separate, collaborators)
collaborators = map(clean, collaborators)
collaborators = list(collaborators)        
collaborations = add_collaborators_to_songs(collaborators, collaborations)

In [87]:
import networkx as nx
G = nx.DiGraph()

In [88]:
from functools import reduce
from operator import add

def unique_artists(songs):
    artists = [song['collaborators'] for song in songs]   # List of lists
    return set(reduce(add, collaborators))

nodes = unique_artists(collaborations)
G.add_nodes_from(nodes)

In [89]:
from typing import List, Tuple

def make_edges(songs) -> List[Tuple]:
    """ Returns tuples, which creates an edge between collaborators """
    edges = []
    for song in songs:
        primary, *collabs = song['collaborators']
        if collabs:
            edges.append((primary, *collabs))
    return edges

def add_edges(graph: nx.Graph, edges: List[Tuple]):
    for primary, *collabs in edges:
        for collab in collabs:
            graph.add_edge(primary, collab)
            
    
edges = make_edges(collaborations)
add_edges(G, edges)

In [90]:
G['Michael Jackson']

AtlasView({'Justin Timberlake': {}, 'The Jacksons': {}, 'Janet Jackson': {}})

In [91]:
import matplotlib.pyplot as plt

plt.subplot(121)
nx.draw(G, with_labels=True, font_weight='bold')
plt.subplot(122)
nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')


NetworkXError: Node '' has no position.

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7fc279a079d8> (for post_execute):


ValueError: 
Joey Bada$$
         ^
Expected end of text (at char 9), (line:1, col:10)

ValueError: 
Joey Bada$$
         ^
Expected end of text (at char 9), (line:1, col:10)

<Figure size 432x288 with 2 Axes>